In [1]:
import argparse
from model.visualglm import VisualGLMModel

[2023-11-30 16:10:20,880] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/qianq/mycodes/VisualGLM-6B/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2023-11-30 16:10:22,216] [WARNING] Failed to load bitsandbytes:No module named 'bitsandbytes'


In [2]:
model, model_args = VisualGLMModel.from_pretrained(
    name="visualglm-6b",
    args=argparse.Namespace(
        fp16=True,
        skip_init=True,
        use_gpu_initialization=True,
        device='cuda',
    )
)
model = model.eval()

[2023-11-30 16:10:22,264] [INFO] building VisualGLMModel model ...
[2023-11-30 16:10:22,293] [INFO] [RANK 0] > initializing model parallel with size 1
[2023-11-30 16:10:22,294] [INFO] [RANK 0] You are using model-only mode.
For torch.distributed users or loading model parallel models, set environment variables RANK, WORLD_SIZE and LOCAL_RANK.
/home/qianq/mycodes/VisualGLM-6B/venv/lib/python3.10/site-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
[2023-11-30 16:10:29,962] [INFO] [RANK 0]  > number of parameters on model parallel rank 0: 7802193408
[2023-11-30 16:10:31,225] [INFO] [RANK 0] global rank 0 is loading checkpoint /home/qianq/.sat_models/visualglm-6b/1/mp_rank_00_model_states.pt
[2023-11-30 16:10:38,085] [INFO] [RANK 0] Will continue but found unexpected_keys! Check whether you are loading correct checkpoints: ['transformer.position_embeddings.weight'].
[2023-11-30 16:10:3

In [3]:
def show_children_struct(model):
    for item in model.named_children():
        print('-----------------')
        print(item[0])
        print(item[1])
        print('-----------------')

def count_model_parameters(model):
    total = sum([param.nelement() for param in model.parameters()])
    print("Number of parameter: %.2fM" % (total/1e6))

In [4]:
show_children_struct(model)

-----------------
mixins
ModuleDict(
  (chatglm-final): ChatGLMFinalMixin(
    (lm_head): ColumnParallelLinear()
  )
  (chatglm-attn): ChatGLMAttnMixin(
    (rotary_emb): RotaryEmbedding()
  )
  (chatglm-layer): ChatGLMLayerMixin()
  (eva): ImageMixin(
    (model): BLIP2(
      (vit): EVAViT(
        (mixins): ModuleDict(
          (patch_embedding): ImagePatchEmbeddingMixin(
            (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
          )
          (pos_embedding): InterpolatedPositionEmbeddingMixin()
          (cls): LNFinalyMixin(
            (ln_vision): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          )
        )
        (transformer): BaseTransformer(
          (embedding_dropout): Dropout(p=0.1, inplace=False)
          (word_embeddings): Embedding(1, 1408)
          (position_embeddings): Embedding(257, 1408)
          (layers): ModuleList(
            (0-38): 39 x BaseTransformerLayer(
              (input_layernorm): LayerNorm((1408,), ep

In [5]:
mixins = model.mixins
show_children_struct(mixins)

-----------------
chatglm-final
ChatGLMFinalMixin(
  (lm_head): ColumnParallelLinear()
)
-----------------
-----------------
chatglm-attn
ChatGLMAttnMixin(
  (rotary_emb): RotaryEmbedding()
)
-----------------
-----------------
chatglm-layer
ChatGLMLayerMixin()
-----------------
-----------------
eva
ImageMixin(
  (model): BLIP2(
    (vit): EVAViT(
      (mixins): ModuleDict(
        (patch_embedding): ImagePatchEmbeddingMixin(
          (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
        )
        (pos_embedding): InterpolatedPositionEmbeddingMixin()
        (cls): LNFinalyMixin(
          (ln_vision): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        )
      )
      (transformer): BaseTransformer(
        (embedding_dropout): Dropout(p=0.1, inplace=False)
        (word_embeddings): Embedding(1, 1408)
        (position_embeddings): Embedding(257, 1408)
        (layers): ModuleList(
          (0-38): 39 x BaseTransformerLayer(
            (input_layernor

In [6]:
eva = model.mixins.eva
show_children_struct(eva)

-----------------
model
BLIP2(
  (vit): EVAViT(
    (mixins): ModuleDict(
      (patch_embedding): ImagePatchEmbeddingMixin(
        (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
      )
      (pos_embedding): InterpolatedPositionEmbeddingMixin()
      (cls): LNFinalyMixin(
        (ln_vision): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
      )
    )
    (transformer): BaseTransformer(
      (embedding_dropout): Dropout(p=0.1, inplace=False)
      (word_embeddings): Embedding(1, 1408)
      (position_embeddings): Embedding(257, 1408)
      (layers): ModuleList(
        (0-38): 39 x BaseTransformerLayer(
          (input_layernorm): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (attention): SelfAttention(
            (query_key_value): ColumnParallelLinear()
            (attention_dropout): Dropout(p=0.1, inplace=False)
            (dense): RowParallelLinear()
            (output_dropout): Dropout(p=0.1, inplace=False)
          )
        

In [7]:
count_model_parameters(eva)

Number of parameter: 1094.26M


In [8]:
vit = eva.model.vit
qformer = eva.model.qformer
count_model_parameters(vit)
count_model_parameters(qformer)

Number of parameter: 985.95M
Number of parameter: 105.16M


In [9]:
show_children_struct(vit)

-----------------
mixins
ModuleDict(
  (patch_embedding): ImagePatchEmbeddingMixin(
    (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
  )
  (pos_embedding): InterpolatedPositionEmbeddingMixin()
  (cls): LNFinalyMixin(
    (ln_vision): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
  )
)
-----------------
-----------------
transformer
BaseTransformer(
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (word_embeddings): Embedding(1, 1408)
  (position_embeddings): Embedding(257, 1408)
  (layers): ModuleList(
    (0-38): 39 x BaseTransformerLayer(
      (input_layernorm): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attention): SelfAttention(
        (query_key_value): ColumnParallelLinear()
        (attention_dropout): Dropout(p=0.1, inplace=False)
        (dense): RowParallelLinear()
        (output_dropout): Dropout(p=0.1, inplace=False)
      )
      (post_attention_layernorm): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (

In [10]:
vit_transformer = vit.transformer
show_children_struct(vit_transformer)

-----------------
embedding_dropout
Dropout(p=0.1, inplace=False)
-----------------
-----------------
word_embeddings
Embedding(1, 1408)
-----------------
-----------------
position_embeddings
Embedding(257, 1408)
-----------------
-----------------
layers
ModuleList(
  (0-38): 39 x BaseTransformerLayer(
    (input_layernorm): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
    (attention): SelfAttention(
      (query_key_value): ColumnParallelLinear()
      (attention_dropout): Dropout(p=0.1, inplace=False)
      (dense): RowParallelLinear()
      (output_dropout): Dropout(p=0.1, inplace=False)
    )
    (post_attention_layernorm): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
    (mlp): MLP(
      (dense_h_to_4h): ColumnParallelLinear()
      (dense_4h_to_h): RowParallelLinear()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)
-----------------


In [13]:
for n, w in vit_transformer.named_parameters():
    print(n)

word_embeddings.weight
position_embeddings.weight
layers.0.input_layernorm.weight
layers.0.input_layernorm.bias
layers.0.attention.query_key_value.weight
layers.0.attention.query_key_value.bias
layers.0.attention.dense.weight
layers.0.attention.dense.bias
layers.0.post_attention_layernorm.weight
layers.0.post_attention_layernorm.bias
layers.0.mlp.dense_h_to_4h.weight
layers.0.mlp.dense_h_to_4h.bias
layers.0.mlp.dense_4h_to_h.weight
layers.0.mlp.dense_4h_to_h.bias
layers.1.input_layernorm.weight
layers.1.input_layernorm.bias
layers.1.attention.query_key_value.weight
layers.1.attention.query_key_value.bias
layers.1.attention.dense.weight
layers.1.attention.dense.bias
layers.1.post_attention_layernorm.weight
layers.1.post_attention_layernorm.bias
layers.1.mlp.dense_h_to_4h.weight
layers.1.mlp.dense_h_to_4h.bias
layers.1.mlp.dense_4h_to_h.weight
layers.1.mlp.dense_4h_to_h.bias
layers.2.input_layernorm.weight
layers.2.input_layernorm.bias
layers.2.attention.query_key_value.weight
layers.2.a

: 